In [3]:
%%time

import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

CPU times: user 775 ms, sys: 149 ms, total: 924 ms
Wall time: 1.1 s


In [4]:
bucket = "sagemaker-studio-208779919433-dm4t8ujq8g"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [5]:
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::208779919433:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
us-east-1
sagemaker-studio-208779919433-dm4t8ujq8g
https://s3-us-east-1.amazonaws.com/sagemaker-studio-208779919433-dm4t8ujq8g


In [6]:
import sklearn
print(sklearn.__version__)

0.22.1


In [7]:
pip install scikit-learn== 1.0.1

ERROR: Ignored the following versions that require a different python version: 1.1.0 Requires-Python >=3.8; 1.1.1 Requires-Python >=3.8; 1.1.2 Requires-Python >=3.8; 1.1.3 Requires-Python >=3.8; 1.2.0 Requires-Python >=3.8; 1.2.0rc1 Requires-Python >=3.8; 1.2.1 Requires-Python >=3.8; 1.2.2 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement scikit-learn== (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21.0, 0.21.1, 0.21.2, 0.21.3, 0.22, 0.22.1, 0.22.2, 0.22.2.post1, 0.23.0, 0.23.1, 0.23.2, 0.24.0, 0.24.1, 0.24.2, 1.0, 1.0.1, 1.0.2)
ERROR: No matching distribution found for scikit-learn==

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
model_file_name = "titanic_GB_model.pkl"

In [9]:
import joblib
import sklearn

model = joblib.load(model_file_name)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DummyClassifier from version 1.0.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 1.0.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [10]:
model

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort=None,
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [11]:
!tar czvf titanic_GB_model.tar.gz $model_file_name

titanic_GB_model.pkl


In [12]:
#### prefix in S3
prefix = "models/titanic"

fObj = open("titanic_GB_model.tar.gz", "rb")
key = os.path.join(prefix,"titanic_GB_model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

models/titanic/titanic_GB_model.tar.gz


In [13]:
model_data = "s3://{}/{}".format(bucket, key)
print(f"model data: {model_data}")

model data: s3://sagemaker-studio-208779919433-dm4t8ujq8g/models/titanic/titanic_GB_model.tar.gz


In [16]:
#scaler = "dg_scaler_v5"
#scaler = joblib.load(scaler)
#scaler

In [14]:
%%writefile script.py

import joblib
import os
from io import StringIO
import numpy as np
import pandas as pd
import sklearn

def predict_fn(input_object, model):
    #prediction = model.predict(input_object)
    pred_prob = model.predict_proba(input_object)
    return pred_prob

def model_fn(model_dir):
    print("loading model.joblib from: {}".format(model_dir))
    loaded_model = joblib.load(os.path.join(model_dir, "titanic_GB_model.pkl"))
    return loaded_model


Writing script.py


In [15]:
from sagemaker.sklearn.model import SKLearnModel
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="1.0-1",
    py_version="py3",
    entry_point="script.py",
)

Create a Model Package Group (Boto3):

In [16]:
import time
import os
from sagemaker import get_execution_role, session
import boto3



sm_client = boto3.client('sagemaker', region_name=region)

Create a Model Package Group for the trained model to be registered:

In [17]:
import time
client = boto3.client("sagemaker")

model_package_group_name = "sample-titanic-models"
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "GB model",
}

create_model_pacakge_group_response = client.create_model_package_group(
    **model_package_group_input_dict
)
model_package_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
print(f"ModelPackageGroup Arn : {model_package_arn}")

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:208779919433:model-package-group/sample-titanic-models


Register the model of the 1st training job in the Model Registry:

In [18]:
inference_instance_type = "ml.m5.xlarge"
model_package_arn = "arn:aws:sagemaker:us-east-1:208779919433:model-package-group/sample-titanic-models"
model_package_1 = model.register(
    model_package_group_name="sample-titanic-models",
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type],
    content_types=["text/csv"],
    response_types=["text/csv"],
    approval_status="Approved",
    description = 'titanic_GB_model'
)

model_package_arn_1 = model_package_1.model_package_arn
print("Model Package ARN : ", model_package_arn_1)

Model Package ARN :  arn:aws:sagemaker:us-east-1:208779919433:model-package/sample-titanic-models/1


In [ ]:
model_package_arn = "arn:aws:sagemaker:us-east-1:208779919433:model-package/sample-titanic-models/1"
from sagemaker import ModelPackage 
from time import gmtime, strftime
sess = sagemaker.Session()

titanic_model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=sess)
sm_model = titanic_model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
   
)

In [22]:
import boto3
import sagemaker
import sagemaker.session


region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = "s3-prd-cssg-mlops"
bucket_int = "sagemaker-studio-453613310276-ldkf3w4xk9"

In [23]:
input_data = "s3://sagemaker-studio-453613310276-ldkf3w4xk9/Globe/scoring-input-data/sg-ug-dg/sg-dg/sg_dg_eval.csv"

In [24]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name="7-2023-03-29-09-31-55-041",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    assemble_with="Line", 
    accept="text/csv",
    max_concurrent_transforms=2,
    max_payload=6,
    strategy="MultiRecord",
    output_path=f"s3://{bucket_int}/Globe/scoring-output-data/sg-ug-dg/sg-dg-1",
)

In [25]:
transformer.transform(data=input_data, split_type="Line", content_type="text/csv")

INFO:sagemaker:Creating transform job with name: 7-2023-03-29-09-31-55-041-2023-03-29-09-36-18-485


.........................2023-03-29 09:40:26,526 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-03-29 09:40:26,526 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-03-29 09:40:26,529 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-03-29 09:40:26,530 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;

UnexpectedStatusException: Error for Transform job 7-2023-03-29-09-31-55-041-2023-03-29-09-36-18-485: Failed. Reason: AlgorithmError: See job logs for more information